In [1]:
import sqlite3
import pandas as pd
import numpy as np
import time
import pickle
import pyarrow.parquet as pq
import pyarrow as pa
import _pickle as cPickle
import feather

In [3]:
# Load the csv for current processing

csvfile = 'hydraulic_systems_demo_data.csv'
df = pd.read_csv(csvfile)

In [4]:
# Time to read csv from pandas

def calculate_read_csv_time(csvfile):
    start = time.time()
    df = pd.read_csv(csvfile)
    csvreadtime = time.time()-start
    return csvreadtime

In [5]:
# Saving in the sqlite db

sqlfile = 'hydraulic.sqlite3'

def calculate_df_to_sql_save_time(csvfile,sqlfile):
    sqlconnection = sqlite3.connect(sqlfile)
    start = time.time()
#   iteration=0
    for chunk in pd.read_csv(csvfile,chunksize=1000):
        chunk.to_sql(name='content',con=sqlconnection,if_exists="append",index=False)
    sqlsavetime = time.time()-start
    return sqlsavetime

In [6]:
# Time to query in the sqlite db
# We are querying for the row where the timestamp is '2020-12-11 03:31:30'

sqlstatement = "SELECT * FROM content WHERE timestamp IS '2020-12-11 03:31:30'"

def calculate_sql_query_time(sqlfile,sqlstatement):
    start = time.time()
    sqlconnection = sqlite3.connect(sqlfile)
    sqlcursor = sqlconnection.cursor()
    sqlcursor.execute(sqlstatement)
    sqlcursor.fetchall()
    sqlquerytime = time.time()-start
    return sqlquerytime

In [7]:
# Time to read the sql db

query = "SELECT * FROM content"
dfsql = None

def calculate_sql_to_df_load_time(query,sqlfile,dfsql):
    start = time.time()
    sqlconnection = sqlite3.connect(sqlfile)
    dfsql = pd.read_sql_query(query,sqlconnection)
    sqlreadtime = time.time()-start
    return sqlreadtime

In [8]:
# Time to save the pickle file

picklfile = "hydraulic.pkl"

def calculate_df_to_pickle_time(picklfile,df):
    start = time.time()
    with open(picklfile,"wb") as f:
        pickle.dump(df,f)
    picklesavetime = time.time()-start
    return picklesavetime

In [9]:
# Time to load the pickle file content as dataframe in pandas

dfpkl = None

def calculate_pickl_to_df_load_time(picklfile,dfpkl):
    start = time.time()
    with open(picklfile,"rb") as f:
        dfpkl = pickle.load(f)
    picklereadtime = time.time()-start
    return picklereadtime

In [10]:
# Time to save the dataframe as hdf5 file
# SKIPPED THIS PORTION WHEN CALCULATING THE TIME AT THE END

hdfile = 'hydraulic.h5'

def calculate_df_to_hdf5_file_time(hdfile,df):
    start = time.time()
    df.to_hdf(hdfile,key='df',mode='w')
    hdfsavetime = time.time()-start
    return dhfsavetime

In [ ]:
# Time to read the hdf5 file
# ERROR IN READING THE FILE OF HDF5 FORMAT

start = time.time()
dfhdf = pd.read_hdf('hydraulic.h5',mode='r+')
hdfreadtime = time.time()-start

In [12]:
# Time to save the dataframe as a parquet file
# Approach 1 - without using pytables

parquet_file1 = 'df.parquet.gzip'
compression_type = 'gzip'

def calculate_df_to_parquet_save_time_approach1(parquet_file,compression_type,df):
    start = time.time()
    df.to_parquet(parquet_file,compression=compression_type)
    parquetsavetime = time.time()-start
    return parquetsavetime

In [13]:
# Time to read the parquet file to the dataframe
# Approach 1

dfparquet1 = None

def calculate_parquet_to_df_load_time_approach1(parquet_file,dfparquet):
    start = time.time()
    dfparquet = pd.read_parquet(parquet_file)
    parquetloadtime = time.time()-start
    return parquetloadtime

In [14]:
# Time to save the parquet file to the dataframe
# Approach 2 - using pytables

parquet_file2 = 'df.parquet'

def calculate_df_to_parquet_save_time_approach2(parquet_file,df):
    start = time.time()
    table = pa.Table.from_pandas(df)
    pq.write_table(table,parquet_file)
    parquetsavetime = time.time()-start
    return parquetsavetime

In [15]:
# Time to read the parquet file to the dataframe
# Approach 2 - using pytables

dfparquet2 = None

def calculate_parquet_to_df_load_time_approach2(parquet_file,dfparquet):
    start = time.time()
    table = pq.read_table(parquet_file)
    dfparquet = table.to_pandas()
    parquetloadtime = time.time()-start
    return parquetloadtime

In [16]:
# Time to save cPickle file

cpklfile = 'hydraulicc.pkl'

def calculate_df_to_cpkl_save_time(cpklfile,df):
    start = time.time()
    with open(cpklfile,"wb") as f:
        cPickle.dump(df,f)
    cpicklesavetime = (time.time()-start)
    return cpicklesavetime

In [17]:
# Time to load cPickle file

dfcpkl = None

def calculate_cpkl_to_df_load_time(cpklfile,dfcpkl):
    start = time.time()
    with open(cpklfile,"rb") as f:
        dfcpkl = cPickle.load(f)
    cpickleloadtime = time.time()-start
    return cpickleloadtime

In [18]:
# Time to save dataframe as feather

featherfile = 'hydraulic.feather'

def calculate_df_to_feather_save_time(featherfile,df):
    start = time.time()
    feather.write_dataframe(df,featherfile)
    feathersavetime = time.time()-start
    return feathersavetime

In [19]:
# Time to read the feather file

dffeather = None

def calculate_feather_to_df_load_time(featherfile,dffeather):
    start = time.time()
    dffeather = feather.read_dataframe(featherfile)
    featherloadtime = time.time()-start
    return featherloadtime

In [20]:
# Time to query the dataframe
# Say we are quering to find the row with timestamp 2020-12-11 03:31:30

def calculate_time_to_query_df(df):
    start = time.time()
    df.loc[df['timestamp'] == "2020-12-11 03:31:30"]
    dfquerytime = time.time()-start
    return dfquerytime

In [ ]:
# Calculating average time calculations

n = 100
read_csv_times = []
df_to_sql_times = []
query_sql_times = []
sql_to_df_times = []
df_to_pkl_times = []
pkl_to_df_times = []
df_to_parquet_app1_times = []
parquet_to_df_app1_times = []
df_to_parquet_app2_times = []
parquet_to_df_app2_times = []
df_to_cpkl_times = []
cpkl_to_df_times = []
df_to_feather_times = []
feather_to_df_times = []
query_df_times = []

for i in range(0,n):
    read_csv_time = calculate_read_csv_time(csvfile)
    read_csv_times.append(read_csv_time)
    
    print("CSV read times:",i+1)
    
    df_to_sql_time = calculate_df_to_sql_save_time(csvfile,sqlfile)
    df_to_sql_times.append(df_to_sql_time)
    
    print("DF to SQL times:",i+1)
    
    query_sql_time = calculate_sql_query_time(sqlfile,sqlstatement)
    query_sql_times.append(query_sql_time)
    
    print("SQL Query times:",i+1)
    
    sql_to_df_time = calculate_sql_to_df_load_time(query,sqlfile,dfsql)
    sql_to_df_times.append(sql_to_df_time)
    
    print("SQL to DF times:",i+1)
    
    df_to_pkl_time = calculate_df_to_pickle_time(picklfile,df)
    df_to_pkl_times.append(df_to_pkl_time)
    
    print("DF to Pickle times:",i+1)
    
    pkl_to_df_time = calculate_pickl_to_df_load_time(picklfile,dfpkl)
    pkl_to_df_times.append(pkl_to_df_time)
    
    print("Pickle to DF times:",i+1)
    
    df_to_parquet_app1_time = calculate_df_to_parquet_save_time_approach1(parquet_file1,compression_type,df)
    df_to_parquet_app1_times.append(df_to_parquet_app1_time)
    
    print("DF to Parquet approach 1 times:",i+1)
    
    parquet_to_df_app1_time = calculate_parquet_to_df_load_time_approach1(parquet_file1,dfparquet1)
    parquet_to_df_app1_times.append(parquet_to_df_app1_time)
    
    print("Parquet to DF approach 1 times:",i+1)
    
    df_to_parquet_app2_time = calculate_df_to_parquet_save_time_approach2(parquet_file2,df)
    df_to_parquet_app2_times.append(df_to_parquet_app2_time)
    
    print("DF to Parquet approach 2 times:",i+1)
    
    parquet_to_df_app2_time = calculate_parquet_to_df_load_time_approach2(parquet_file2,dfparquet2)
    parquet_to_df_app2_times.append(parquet_to_df_app2_time)
    
    print("Parquet to DF approach 2 times:",i+1)
    
    df_to_cpkl_time = calculate_df_to_cpkl_save_time(cpklfile,df)
    df_to_cpkl_times.append(df_to_cpkl_time)
    
    print("DF to cPickle times:",i+1)
    
    cpkl_to_df_time = calculate_cpkl_to_df_load_time(cpklfile,dfcpkl)
    cpkl_to_df_times.append(cpkl_to_df_time)
    
    print("cPickle to DF times:",i+1)
    
    df_to_feather_time = calculate_df_to_feather_save_time(featherfile,df)
    df_to_feather_times.append(df_to_feather_time)
    
    print("DF to feather times:",i+1)
    
    feather_to_df_time = calculate_feather_to_df_load_time(featherfile,dffeather)
    feather_to_df_times.append(feather_to_df_time)
    
    print("Feather to DF times:",i+1)
    
    query_df_time = calculate_time_to_query_df(df)
    query_df_times.append(query_df_time)
    
    print("Query DF times:",i+1)

CSV read times: 1


/Users/quartic/quarticai/learn-projects/lenv/lib/python3.6/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


DF to SQL times: 1
SQL Query times: 1
SQL to DF times: 1
DF to Pickle times: 1
Pickle to DF times: 1
DF to Parquet approach 1 times: 1
Parquet to DF approach 1 times: 1
DF to Parquet approach 2 times: 1
Parquet to DF approach 2 times: 1
DF to cPickle times: 1
cPickle to DF times: 1
DF to feather times: 1
Feather to DF times: 1
Query DF times: 1
CSV read times: 2
DF to SQL times: 2
SQL Query times: 2
SQL to DF times: 2
DF to Pickle times: 2
Pickle to DF times: 2
DF to Parquet approach 1 times: 2
Parquet to DF approach 1 times: 2
DF to Parquet approach 2 times: 2
Parquet to DF approach 2 times: 2
DF to cPickle times: 2
cPickle to DF times: 2
DF to feather times: 2
Feather to DF times: 2
Query DF times: 2
CSV read times: 3
DF to SQL times: 3
SQL Query times: 3
SQL to DF times: 3
DF to Pickle times: 3
Pickle to DF times: 3
DF to Parquet approach 1 times: 3
Parquet to DF approach 1 times: 3
DF to Parquet approach 2 times: 3
Parquet to DF approach 2 times: 3
DF to cPickle times: 3
cPickle to

In [ ]:
# Hashing the pickle
import hmac, hashlib
pickled_data = pickle.dumps(df)
digest = hmac.new('key',pickled_data,hashlib.sha1).hexdigest()
header = '%s' % (digest)